# NetCDF zu GeoTIFF Konvertierung
Dieses Skript lädt NetCDF-Dateien, extrahiert relevante Daten, und speichert sie als GeoTIFF-Dateien. Da NetCDF-Dateien oft Zeitschritte enthalten, wird das Skript die Daten über die Zeitachsen iterieren und mehrere TIFF-Dateien erzeugen (eines pro Zeitschritt).
Die Struktur der NetCDF-Dateien wird analysiert, und relevante Informationen werden in ein Logfile geschrieben. 


### Bibliotheken:
- `xarray`: Für das Laden und Bearbeiten von NetCDF-Daten.
- `rasterio`: Für die Erstellung von GeoTIFF-Dateien.
- `numpy`: Für die Arbeit mit numerischen Daten.

### Vorgehen:
1. Bibliotheken instllieren, laden und NetCDF-Datei öffnen.
2. Überprüfen der Datenstruktur (Dimensionen, Variablen).
3. Konvertierung der NetCDF-Daten (pro Zeitschritt) in GeoTIFF.

# Verarbeitung von NetCDF-Dateien
In diesem Notebook werden NetCDF-Dateien verarbeitet, um Niederschlagsdaten zu extrahieren und in das GeoTIFF-Format zu konvertieren. Der Benutzer kann entweder spezifische Zeitschritte angeben oder alle Zeitschritte auf einmal verarbeiten. 
Die Ergebnisse werden in einem festgelegten Ausgabeverzeichnis gespeichert, und eine Logdatei dokumentiert die Struktur der ersten Datei.

In [4]:
# Funktion zur Prüfung und Installation von Modulen
import sys
import subprocess

def install_and_import(package):
    try:
        __import__(package)
    except ImportError:
        print(f"{package} ist nicht installiert. Installation wird durchgeführt...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
    finally:
        globals()[package] = __import__(package)

# Liste der benötigten Module
modules = ['xarray', 'rasterio', 'numpy', 'glob', 'os', 'packaging', 'pandas', 'netCDF4', 'h5netcdf', 'h5py', 'scipy']

# Prüfen und ggf. Installieren der Module
for module in modules:
    install_and_import(module)


In [5]:
# Bibliotheken importieren
from sympy import im
import xarray as xr
import rasterio
import numpy as np
import os
import packaging
import pandas
import netCDF4
import h5netcdf
import h5py
import scipy
import glob


## 2. Funktionen zur Protokollierung und Datenverarbeitung

In dieser Sektion definieren wir die Funktionen zur Protokollierung der Struktur der Datensätze, zur Protokollierung der gefundenen Dateien und deren Zeitschritte sowie zur Verarbeitung und zum Export der GeoTIFF-Dateien.


In [6]:
# Zelle 2: Funktion zur Protokollierung der Struktur und Attribute einer Dataset-Datei
def log_dataset_structure(ds, log_file, nc_file):
    with open(log_file, 'a') as f:
        f.write(f"=== Struktur der Datei: {os.path.basename(nc_file)} ===\n")  # Logge den Dateinamen
        f.write(f"Dataset Struktur:\n")
        f.write(str(ds) + "\n\n")  # Schreibe die Struktur des Datensatzes
        f.write(f"Attribute:\n")
        for attr in ds.attrs:
            f.write(f"{attr}: {ds.attrs[attr]}\n")  # Schreibe die Attribute des Datensatzes
        f.write("\n" + "="*40 + "\n\n")

# Zelle 3: Funktion zur Protokollierung der gefundenen Dateien und deren Zeitschritte
def log_found_files_and_time_steps(nc_files, log_file):
    with open(log_file, 'a') as f:
        f.write("=== Gefundene Dateien und deren Zeitschritte ===\n")
        for nc_file in nc_files:
            ds = xr.open_dataset(nc_file, engine='netcdf4')  # Lade die Datei, um die Zeitschritte zu ermitteln
            time_steps = ds['time'].values
            f.write(f"Datei: {os.path.basename(nc_file)} - Zeitschritte: {len(time_steps)}\n")  # Logge die Anzahl der Zeitschritte
        f.write("\n" + "="*40 + "\n\n")

# Zelle 4: Funktion zum Exportieren von GeoTIFF-Dateien
def export_to_geotiff(time_index, ds, crs_value, transform, nc_file):
    output_file = f"./output/{os.path.basename(nc_file).replace('.', '_')}_time{time_index}.tif"  # Ersetze '.' durch '_' im Dateinamen
    with rasterio.open(
        output_file,
        'w',
        driver='GTiff',
        height=ds.dims['y'],
        width=ds.dims['x'],
        count=1,
        dtype=ds['RR'].dtype,
        crs=crs_value,
        transform=transform
    ) as dst:
        dst.write(ds['RR'].isel(time=time_index).values, 1)  # Schreibe die Daten in die GeoTIFF-Datei
    return output_file

# Zelle 5: Funktion zur Verarbeitung aller .nc Dateien im Ordner
def process_all_nc_files(nc_files, time_step_input, log_file):
    first_ds_logged = False  # Flag, um zu prüfen, ob die Struktur der ersten Datei geloggt wurde
    total_files_processed = 0  # Zähler für die verarbeiteten Dateien
    output_files = []  # Liste für die erstellten Ausgabedateien
    errors = []  # Liste für Fehler

    log_found_files_and_time_steps(nc_files, log_file)  # Protokolliere die gefundenen Dateien und deren Zeitschritte

    for nc_file in nc_files:
        print(f"Verarbeite Datei: {nc_file}")
        try:
            # Logge die Struktur des ersten Datensatzes
            if not first_ds_logged:
                ds = xr.open_dataset(nc_file, engine='netcdf4')  # Lade die aktuelle Datei
                log_dataset_structure(ds, log_file, nc_file)  # Protokolliere die Struktur der ersten Datei
                first_ds_logged = True
            else:
                ds = xr.open_dataset(nc_file, engine='netcdf4')  # Lade die aktuelle Datei

            selected_time_steps = parse_time_steps(time_step_input, ds)  # Bestimme die zu exportierenden Zeitschritte
            crs_value = ds['crs'].attrs.get('proj4', 'Keine gültige CRS-Information im proj4-Format vorhanden')  # Überprüfe CRS
            transform = get_transform(ds)  # Berechne den Transformationsparameter

            # Exportiere die Daten für jeden ausgewählten Zeitschritt
            for time_index in selected_time_steps:
                output_file = export_to_geotiff(time_index, ds, crs_value, transform, nc_file)  # Exportiere die GeoTIFF-Datei
                output_files.append(output_file)  # Logge den Namen der erstellten Datei
            
            total_files_processed += 1  # Erhöhe den Zähler für die verarbeiteten Dateien

        except Exception as e:
            errors.append(f"Fehler bei der Verarbeitung von {nc_file}: {str(e)}")  # Protokolliere den Fehler

    # Protokolliere die Ergebnisse am Ende
    with open(log_file, 'a') as f:
        f.write("=== Verarbeitungsergebnisse ===\n")
        f.write(f"Anzahl verarbeiteter Dateien: {total_files_processed}\n")
        for output_file in output_files:
            f.write(f"Erstellte Datei: {output_file}\n")
        if errors:
            f.write("=== Fehler ===\n")
            for error in errors:
                f.write(f"{error}\n")
        f.write("\n" + "="*40 + "\n\n")


## 3. Hilfsfunktionen

Hier definieren wir die Hilfsfunktionen zum Parsen der Zeitschritte und zur Berechnung der Transformationsparameter.


In [7]:
# Zelle 6: Hilfsfunktionen zum Parsen der Zeitschritte und zum Berechnen der Transformationsparameter
def parse_time_steps(time_step_input, ds):
    if time_step_input == 'all':
        return range(len(ds['time'].values))  # Alle Zeitschritte
    else:
        time_indices = []
        for item in time_step_input.split(','):
            if '-' in item:
                start, end = map(int, item.split('-'))
                time_indices.extend(range(start, end + 1))
            else:
                time_indices.append(int(item))
        return time_indices

def get_transform(ds):
    # Berechnung des Transformationsparameters für Rasterdaten
    transform = rasterio.transform.from_origin(
        ds['lon'].min().item(),  # Westliche Grenze
        ds['lat'].max().item(),  # Nördliche Grenze
        (ds['lon'].max() - ds['lon'].min()) / ds.dims['x'],  # Pixelgröße in x-Richtung
        (ds['lat'].max() - ds['lat'].min()) / ds.dims['y']   # Pixelgröße in y-Richtung
    )
    return transform


## 4. Anwendung und Ausführung

In dieser Sektion geben wir den Pfad zum Verzeichnis mit den NetCDF-Dateien an, definieren die zu verarbeitenden Zeitschritte und starten den Verarbeitungsprozess.


In [ ]:
import glob

# Zelle 7: Eingabe des Verzeichnisses und der zu verarbeitenden Zeitschritte
# Ordner für die Eingabedateien
input_directory = './data/01/'  # Geben Sie hier den Pfad zu Ihrem Datenverzeichnis an
nc_files = glob.glob(os.path.join(input_directory, '**/*.nc'), recursive=True)  # Suche nach allen .nc Dateien im Verzeichnis

# Zelle 8: Benutzerdefinierte Eingabe für die Zeitschritte
time_step_input = input("Geben Sie die zu exportierenden Zeitschritte ein (z.B. 'all', '0-5', '10,15'): ")  # Standard ist 'all'
log_file_path = './output/log.txt'  # Pfad zum Logfile

# Zelle 9: Verarbeiten aller .nc Dateien
process_all_nc_files(nc_files, time_step_input, log_file_path)  # Starte die Verarbeitung


### Fazit

Dieses Notebook bietet eine umfassende Möglichkeit zur Verarbeitung von NetCDF-Daten und deren Export in ein für GIS-Anwendungen geeignetes Format. Alle relevanten Informationen werden in ein Logfile geschrieben, um die Nachvollziehbarkeit der Datenverarbeitung zu gewährleisten.


# =================================================================

In [ ]:
# Extrahiere die 'crs'-Information und zeige sie an
if 'crs' in ds.variables:
    crs_value = ds['crs'].attrs.get('proj4', 'Keine gültige CRS-Information im proj4-Format vorhanden')
    print(f"CRS-Proj4-String: {crs_value}")
else:
    crs_value = None
    print("Keine CRS-Variable gefunden.")

## Schritt 2: Auswahl der Variable und des Zeitintervalls
Wähle die Variable aus der NetCDF-Datei, die du konvertieren möchtest (z.B. Temperatur, Niederschlag, etc.), und definiere den Zeitschritt, den du in eine GeoTIFF-Datei umwandeln möchtest.


In [ ]:
import re

# Zeitschrittauswahl (manuelle Eingabe)
# Standard ist "all", aber kann auf spezifische Zeitschritte oder Bereiche angepasst werden.
time_step_input = "all"  # Beispiel: "0-10, 15, 20-25"
time_step_input = "0-3"

# Funktion zur Umwandlung der Eingabe in eine Liste von Zeitschritten
def parse_time_steps(input_str, ds):
    if input_str.strip().lower() == "all":
        return list(range(len(ds['time'])))
    else:
        time_steps = []
        ranges = input_str.split(",")
        for r in ranges:
            if "-" in r:
                start, end = map(int, r.split("-"))
                time_steps.extend(range(start, end + 1))
            else:
                time_steps.append(int(r))
        return time_steps

# Verarbeite die Eingabe der Zeitschritte
selected_time_steps = parse_time_steps(time_step_input, ds)

# Falls keine gültige Auswahl getroffen wurde
if not selected_time_steps:
    print("Keine gültige Auswahl getroffen. Vorgang wird abgebrochen.")
else:
    print(f"Ausgewählte Zeitschritte: {selected_time_steps}")


In [ ]:
import rasterio
from rasterio.transform import from_origin
import pandas as pd
import re

# Funktion zur Bereinigung des Dateinamens
def clean_filename(filename):
    # Ersetze Punkte (.) im Dateinamen durch Unterstriche, außer der letzten für die Dateiendung
    cleaned_name = re.sub(r'\.', '_', filename[:filename.rfind('.')]) + filename[filename.rfind('.'):]
    return cleaned_name

# Metadaten (Transform) für die GeoTIFF-Erstellung
transform = from_origin(min(ds['lon'].values.flatten()), max(ds['lat'].values.flatten()), abs(ds['x'][1] - ds['x'][0]), abs(ds['y'][1] - ds['y'][0]))

# Funktion zur Konvertierung eines Zeitschritts in ein GeoTIFF
def export_to_geotiff(time_index, ds, crs_value, transform, nc_file_path):
    rr_data = ds['RR'].isel(time=time_index)  # Niederschlagsdaten für den Zeitschritt
    timestamp = pd.to_datetime(str(ds['time'].values[time_index]))  # Zeitstempel
    
    # Dateinamen generieren
    nc_filename = os.path.basename(nc_file_path).split('.')[0]
    cleaned_nc_filename = clean_filename(nc_filename)
    variable_name = 'RR'  # Variable, die exportiert wird
    output_filename = f"{cleaned_nc_filename}_{variable_name}_{timestamp.strftime('%Y%m%d_%H%M%S')}.tif"
    
    output_tiff = f'./output/{output_filename}'
    
    # Schreibe die Daten in eine GeoTIFF-Datei
    with rasterio.open(
        output_tiff, 'w',
        driver='GTiff',
        height=rr_data.shape[0],
        width=rr_data.shape[1],
        count=1,
        dtype=rr_data.dtype,
        crs=crs_value,  # Verwende die CRS-Information aus der Datei
        transform=transform
    ) as dst:
        dst.write(rr_data.values, 1)
    
    print(f"GeoTIFF für Zeitschritt {time_index} mit Zeitstempel {timestamp} wurde erfolgreich unter {output_tiff} gespeichert.")

# Iteriere über die ausgewählten Zeitschritte und exportiere sie als GeoTIFF
for time_index in selected_time_steps:
    export_to_geotiff(time_index, ds, crs_value, transform, nc_file_path)


## Schritt 3: Konvertierung in GeoTIFF
Nun konvertieren wir die extrahierten Rasterdaten in eine GeoTIFF-Datei. Dafür wird die Bibliothek `rasterio` verwendet. Wir müssen auch die Geoinformationen wie die Projektion (z.B. WGS84) und die Bounding Box bereitstellen.


In [ ]:
# Erstellen eines GeoTIFF-Dateipfads
output_tif = 'output_raster.tif'

# Beispielhafte affine Transformation und CRS (proj4 oder EPSG-Code)
transform = rasterio.transform.from_origin(west=0, north=0, xsize=0.1, ysize=0.1)  # Beispielhafte Parameter
crs = 'EPSG:4326'  # WGS84

# Öffnen einer neuen TIFF-Datei zum Schreiben
with rasterio.open(
    output_tif, 
    'w', 
    driver='GTiff', 
    height=data_at_time.shape[0], 
    width=data_at_time.shape[1], 
    count=1, 
    dtype=str(data_at_time.dtype), 
    crs=crs, 
    transform=transform
) as dst:
    # Schreibe die Daten in die TIFF-Datei
    dst.write(data_at_time.values, 1)

print(f'Datei {output_tif} erfolgreich erstellt.')


## Schritt 4: Konvertierung für alle Zeitschritte (optional)
Falls die NetCDF-Datei mehrere Zeitschritte enthält und du alle in separate TIFF-Dateien konvertieren möchtest, kannst du über die Zeitachse iterieren und für jeden Zeitschritt eine TIFF-Datei erzeugen.


In [ ]:
# Erstellen eines Ordners zum Speichern der GeoTIFFs
output_folder = 'output_tiffs'
os.makedirs(output_folder, exist_ok=True)

# Über die Zeitachsen iterieren
for i in range(len(ds.coords['time'])):
    # Extrahiere Daten für den aktuellen Zeitschritt
    data_at_time = data_var.isel(time=i)
    
    # Dateiname für den aktuellen Zeitschritt
    output_tif = os.path.join(output_folder, f'raster_time_{i}.tif')
    
    # GeoTIFF-Datei erstellen
    with rasterio.open(
        output_tif, 
        'w', 
        driver='GTiff', 
        height=data_at_time.shape[0], 
        width=data_at_time.shape[1], 
        count=1, 
        dtype=str(data_at_time.dtype), 
        crs=crs, 
        transform=transform
    ) as dst:
        dst.write(data_at_time.values, 1)

    print(f'Datei {output_tif} für Zeitschritt {i} erfolgreich erstellt.')


## Fazit
Das Skript ermöglicht die Konvertierung von NetCDF-Daten in GeoTIFF-Dateien, wobei Zeitschritte berücksichtigt werden. Diese Methode kann verwendet werden, um rasterbasierte Zeitreihenanalysen in einem georeferenzierten Format (TIFF) zu speichern und weiterzuverarbeiten.
